In [32]:
import gmaps
import pandas as pd
import requests
from config import google_key
import matplotlib.pyplot as plt

In [33]:
weather_data=pd.read_csv("Data/weatherpy.csv")
gmaps.configure(api_key=google_key)

In [37]:
# Ask the customer to add a minimum and maximum temperature value.
mintemp = float(input("What is the minimum temperature you would like for your trip? "))
maxtemp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 100
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [38]:
# Filter the dataset to find the cities that fit the criteria.
preferred_temp_df = weather_data.loc[(weather_data["Max Temp"] <= maxtemp) & (weather_data["Max Temp"] >= mintemp)]
if rain == 'yes':
    preferred_precipitation= preferred_temp_df.loc[(preferred_temp_df['Rainfall']>0.)]
else:
    preferred_precipitation= preferred_temp_df.loc[(preferred_temp_df['Rainfall']==0.)]
if snow == 'yes':
    preferred_cities=preferred_precipitation.loc[preferred_precipitation['Snowfall']>0.]
else:
    preferred_cities=preferred_precipitation.loc[preferred_precipitation['Snowfall']==0.]

In [39]:
len(preferred_cities)

56

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Current Conditions", "Lat", "Lng","Max Temp"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Current Conditions,Lat,Lng,Max Temp,Hotel Name
14,Lagoa,PT,light rain,39.05,-27.98,56.95,
23,Hithadhoo,MV,light rain,-0.60,73.08,83.16,
33,Bengkulu,ID,moderate rain,-3.80,102.27,78.73,
35,Hermanus,ZA,light rain,-34.42,19.23,64.00,
44,Meulaboh,ID,light rain,4.14,96.13,80.47,
54,Tevriz,RU,light rain,57.51,72.40,36.52,
80,Tukrah,LY,light rain,32.53,20.58,61.99,
81,Itarema,BR,light rain,-2.92,-39.92,84.07,
95,Ipora,BR,light rain,-16.44,-51.12,83.07,
99,Kudahuvadhoo,MV,light rain,2.67,72.89,84.15,


In [15]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": google_key,
    "location": " "}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print ('Found')
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
hotel_df.to_csv("Data/WeatherPy_vacation.csv", index_label="City_ID")

In [23]:
info_box_template = '''
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Conditions} and {Max Temp} °F</dd>
</dl>
'''
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Add marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))